# AGiXT Enterprise Back End Tests

Initialize XT Systems SDK and run tests for the back end.

## Register a user

Register a user on the auth server.

In [ ]:
import random
import string
import time
from xts import XTSystemsSDK


time.sleep(60)

failures = 0
random_string = "".join(
    random.choices(string.ascii_uppercase + string.digits, k=10)
).lower()
test_email = f"{random_string}@xt.systems"
xts = XTSystemsSDK()

while failures < 100:
    try:
        otp_uri = xts.register_user(
            email=test_email, first_name="Test", last_name="User"
        )
        break
    except Exception as e:
        print(e)
        failures += 1
        time.sleep(5)

# Show QR code for MFA setup
import qrcode
from IPython.display import Image

qr = qrcode.QRCode()
qr.add_data(otp_uri)
qr.make(fit=True)
img = qr.make_image(fill="black", back_color="white")
img.save("qr.png")
Image(filename="qr.png")

## Confirm user exists


In [4]:
user_exists = xts.user_exists(email=test_email)

Status Code: 200
Response JSON:
True


## Update User's Last Name

In [5]:
update_user = xts.update_user(first_name="Super", last_name="Man")

Status Code: 200
Response JSON:
{'detail': 'User updated successfully.'}


## Get User Details

In [6]:
user_details = xts.get_user()

Status Code: 200
Response JSON:
{'email': '9rf4j054d2@devxt.com',
 'first_name': 'Super',
 'input_tokens': '0',
 'ip_address': '127.0.0.1',
 'last_name': 'Man',
 'output_tokens': '0',
 'timezone': ''}


In [ ]:
import openai

openai.base_url = f"{xts.base_url}/v1/"
openai.api_key = xts.headers["Authorization"]


response = openai.chat.completions.create(
    model="XT",
    messages=[
        {
            "role": "user",
            "content": "How would I run the project?",
        }
    ],
    user="Time to test",
)

print(response.choices[0].message.content)

In [ ]:
conversation = xts.get_conversation(conversation_name="Time to test")
print(conversation)

In [ ]:
conversations = xts.get_conversations()
print(conversations)

## Invite a User to a Company

In [22]:
invitation = xts.create_invitation(
    email="invited_user@example.com",
    company_id="",
    role_id=3,  # Assuming 3 is the role_id for a regular user
)

Status Code: 200
Response JSON:
{'company_id': '12d6ddcf-9100-4a6a-b4f2-ea31122996e8',
 'created_at': '2024-08-06T16:34:38',
 'email': 'invited_user@example.com',
 'id': '95df5436-97b7-4275-b6ea-048480aac27b',
 'inviter_id': '2a330d6d-18cb-43b3-aef2-1490beb14d07',
 'is_accepted': False,
 'role_id': 3}


## Accept Invitation

In [23]:
accept_invitation = xts.accept_invitation(invitation_id=invitation["id"])

Status Code: 200
Response JSON:
True


## Delete User

In [40]:
delete_user_result = xts.delete_user()

Status Code: 200
Response JSON:
{'detail': 'User deleted successfully.'}
